<a href="https://colab.research.google.com/github/jeffheaton/present/blob/master/WUSTL/CABI-Demand/lab-3-cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Washington University [Olin School of Business](https://olin.wustl.edu/EN-US/Pages/default.aspx)
[Center for Analytics and Business Insights](https://olin.wustl.edu/EN-US/Faculty-Research/research-centers/center-analytics-business-insights/Pages/default.aspx) (CABI)  
[Deep Learning for Demand Forecasting](https://github.com/jeffheaton/present/tree/master/WUSTL/CABI-Demand)  
Copyright 2022 by [Jeff Heaton](https://www.youtube.com/c/HeatonResearch), Released under [CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/)  


# Lab 3: Computer Vision

Attempt to add one or more of the following features from the data.

* Count people on beach and sidewalks separately.

You can use the following starter code. 
Connect GDrive, to write out any results.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Load the three data files.

In [ ]:
import pandas as pd
import os

PATH = "/content/drive/MyDrive/projects/demand/"

df_sales = pd.read_csv("https://data.heatonresearch.com/wustl/CABI/demand-forecast/sales_train.csv", parse_dates=['date'])
df_items = pd.read_csv("https://data.heatonresearch.com/wustl/CABI/demand-forecast/items.csv")
df_resturant = pd.read_csv("https://data.heatonresearch.com/wustl/CABI/demand-forecast/resturants.csv")

### Prerequisite from Example

In [ ]:
!wget https://data.heatonresearch.com/wustl/CABI/demand-forecast/street-20221003.zip /content/
!unzip /content/street-20221003.zip > /dev/null

import sys

!git clone https://github.com/ultralytics/yolov5 --tag 6.2  # clone
!mv /content/6.2 /content/yolov5
%pip install -qr /content/yolov5/requirements.txt  # install
sys.path.insert(0,'/content/yolov5')

import torch
import utils
display = utils.notebook_init()  # checks

!rm -R /content/yolov5/runs/detect/*
!mkdir /content/images
!cp /content/street/2020_01_01.jpg /content/images

from IPython.display import Image

!python /content/yolov5/detect.py --weights yolov5s.pt --img 1024 \
  --conf 0.25 --source /content/images/

URL = '/content/yolov5/runs/detect/exp/2020_01_01.jpg'
Image(filename=URL, width=300)

### Demonstrate YOLOv5



In [ ]:
import torch

# Model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5n - yolov5x6, custom

# Images
img = '/content/street/2020_01_01.jpg'  # or file, Path, PIL, OpenCV, numpy, list

# Inference
results = yolo_model(img)

# Results
df = results.pandas().xyxy[0]
df

### Extract Data from Street Images

In [ ]:
import os
from os import walk
import datetime
import tqdm
import pandas as pd

STREET_PATH = "/content/street/"
å
filenames = next(walk(STREET_PATH), (None, None, []))[2]  

list_date = []
list_people = []

for file in tqdm.tqdm(filenames):
  filename = os.path.join(STREET_PATH, file)
  results = yolo_model(filename)
  df = results.pandas().xyxy[0]
  people = len(df[df.name=='person'])
  dt = datetime.datetime.strptime(file[:10], '%Y_%m_%d')
  list_date.append(dt)
  list_people.append(people)

df_street_view = pd.DataFrame({'date':list_date,'people':list_people})
df_street_view